<a href="https://colab.research.google.com/github/nikitha-ramchander/sales-prediction/blob/main/SalesDataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Imports 
import pandas as pd
import numpy as np

In [3]:
# Load Data
sales = pd.read_csv('/content/drive/MyDrive/Coding Dojo/Project/salespredictions.csv')
sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


# Irrelevant Data
Checked to see if there is information unnecessary or unrelated to my task. 

In [4]:
# Checking all the columns 
sales.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228


There are no columns to drop because all columns are useful to my analysis. 

#Data Types
You want you data to all be correct data types. Checked the datatypes & validated that they are what they should be. 

In [5]:
# Any duplicated rows?
sales.duplicated().any()

False

In [6]:
# Are all data types correct?
sales.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [7]:
# Are there missing values? 
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


There are missing values in columns Item_Weight and Outlet_Size! We will come back to this later.

#Syntax or Spelling Errors 
Check to see if there are typos or inconsistencies in strings that need to be fixed.

In [8]:
# Are strings in Item_Fat_Content accurate? 
sales['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [10]:
# There are multiple strings that can fit into two categories
# Let's create a dictionary
sales['Item_Fat_Content'] = sales['Item_Fat_Content'].map({'LF': 'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat', 'Low Fat': 'Low Fat', 'Regular': 'Regular'})

In [11]:
# Check to see if it worked
sales['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [12]:
# Are strings in Item_Type accurate? 
sales['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

#Missing Values
Checking for missing data and determining the best way to deal with this.

In [13]:
# Lets create an extra column to flag all the missing values in the Item Weight column
# Later we can see if my changes affect Linear Regression or KNN
# Collaborated with Cassandra 
sales['No_Weight_Flag'] = np.where(sales['Item_Weight'].isnull(), 1, 0)

In [14]:
sales.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
No_Weight_Flag                  0
dtype: int64

Let's first tackle the Outlet Size column for all missing values.

>When looking at the Outlet Size column, I played around with the data to find any correlations with other columns. This way I could assume and input the correct missing value. 

>I noticed all the missing values for Outlet Size fell under 'Tier 2' or 'Tier 3' in Outlet Location Type. 'Tier 1' had no missing values, only values of 'Small' or 'Medium'. For 'Tier 2' it had values for 'Small' and missing values. Lastly, 'Tier 3' had values for 'Medium' and missing values. With that I infer that values missing for 'Tier 3' should be filled in with 'High' Outlet Location Type and values missing for 'Tier 2' should be filled in with 'Medium' Outlet Location Type. 

In [17]:
# Lets filter out by Tier 2 and Tier 3 to fill in missing values for Outlet Size
# Collaborated with Cassandra
Tier2_Filter = sales['Outlet_Location_Type'] == 'Tier 2'
Tier3_Filter = sales['Outlet_Location_Type'] == 'Tier 3'
OutletSize_Filter = sales['Outlet_Size'].isnull()

In [18]:
# The missing values for Tier 2 will be Medium
sales.loc[Tier2_Filter & OutletSize_Filter, 'Outlet_Size'] = 'Medium'

In [19]:
# The missing values for Tier 3 will be High
sales.loc[Tier3_Filter & OutletSize_Filter, 'Outlet_Size'] = 'High'

In [20]:
# Check to see if it worked
sales.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
No_Weight_Flag                  0
dtype: int64

Next lets tackle the Item Weight column for all missing values. 
> It wouldn't make sense to fill Item Weight missing values with '0'. I decided to take the average Item Weight for each Item Type and fill in the missing values with those numbers.

In [21]:
# I want to take a look at the mean Item Weight for each Item Type
sales.groupby(['Item_Type'])[['Item_Weight']].mean()

,Item_Weight
Item_Type,
Baking Goods,12.277108
Breads,11.346936
Breakfast,12.768202
Canned,12.305705
Dairy,13.426069
Frozen Foods,12.867061
Fruits and Vegetables,13.224769
Hard Drinks,11.400328
Health and Hygiene,13.142314


In [22]:
# Replaced missing values of Item Weight with mean values for each Item Type 
# https://stackoverflow.com/questions/40299055/pandas-how-to-fill-null-values-with-mean-of-a-groupby
sales['Item_Weight'] = sales['Item_Weight'].fillna(sales.groupby('Item_Type')['Item_Weight'].transform('mean'))

In [24]:
# Let's see if it works 
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  No_Weight_Flag             8523 non-null   int64  
dtypes: float64(4), int64(2), object(7)
memory usage:

Great we have filled in all the missing values!

#We have finished Data Cleaning, let's move onto Data Visualization. 

In [26]:
sales.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,No_Weight_Flag
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,0
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,0


In [27]:
sales.to_csv('/content/drive/MyDrive/Coding Dojo/Project/salesdatacleaning.csv', index= None)